In [1]:
import lpips
loss_fn_alex = lpips.LPIPS(net='alex') # best forward scores
loss_fn_vgg = lpips.LPIPS(net='vgg') # closer to "traditional" perceptual loss, when used for optimization

import torch
img0 = torch.zeros(1,3,64,64) # image should be RGB, IMPORTANT: normalized to [-1,1]
img1 = torch.zeros(1,3,64,64)
d = loss_fn_alex(img0, img1)

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /Users/minha/dev/tinova/ai-copyright-portection/.venv/lib/python3.9/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /Users/minha/.cache/torch/hub/checkpoints/vgg16-397923af.pth


100%|██████████| 528M/528M [00:45<00:00, 12.0MB/s] 


Loading model from: /Users/minha/dev/tinova/ai-copyright-portection/.venv/lib/python3.9/site-packages/lpips/weights/v0.1/vgg.pth


In [6]:
from PIL import Image
import torchvision.transforms as transforms

# 이미지 로드 및 전처리
def load_image(path):
    image = Image.open(path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize([0.5]*3, [0.5]*3)  # LPIPS는 [-1, 1] 입력 필요
    ])
    return transform(image).unsqueeze(0)

img_path1 = load_image("../data/images/dataset/ex1.jpeg")
img_path2 = load_image("../data/images/dataset/ex2.jpeg")

# 유사도 계산
with torch.no_grad():
    dist = loss_fn_alex(img_path1, img_path2)

print("LPIPS distance:", dist.item())  # 0에 가까울수록 유사함

LPIPS distance: 0.373117595911026


| LPIPS 점수     | 유사도 평가 | 비고                          |
| ------------ | ------ | --------------------------- |
| 0.00 \~ 0.20 | 매우 유사  | 구도, 색감, 질감까지 거의 동일한 경우 많음   |
| 0.20 \~ 0.40 | 부분 유사  | 구성이나 스타일 유사하지만 세부 디테일 차이 있음 |
| 0.40 \~ 0.70 | 낮은 유사도 | 대체로 다른 그림으로 간주됨             |
| 0.70 \~ 1.00 | 거의 무관  | 전혀 다른 이미지                   |
